NOMBRE: Jose Miguel Gonzalez


CARNE: 20335


FECHA:10/10/2024


**Objetivo**: La idea de esta práctica es que exploren el desarrollo de un proceso de cuatización. Para esto, deben cubrir lo que se vio en clase:
1. Efectuar el proceso de cuantización.
2. Determinar su error.

In [1]:
from torch import tensor
import torch

In [2]:
#Así como se vio en clase, definan una proceso de linear quantization para test_tensor
#Para llevar esto a acabo, debe encontrar la escala: scale = rmax-rmin/qmax-qmin.
#Trabajando con pytorch int8 (para mapear a int4) qmax=127 y qmin=128.
#zero_point = qmin-(rmin/scale).
#Con esta informacion ya pueden proceder a trabaja la cuantización lineal.

# Tensor inicial
input_tensor = torch.tensor([[-0.6870,  0.2607, -0.7718,  0.0841],
                             [ 0.6558, -1.1711, -1.0713,  0.2405],
                             [ 1.4967, -0.6928,  0.7961, -1.4614],
                             [ 0.5689, -0.5227, -1.3111, -0.8343]])

target_quantized = torch.tensor([[-77, -77, -77, -77],
                                 [-77, -77, -77, -77],
                                 [-77, -77, -77, -77],
                                 [-77, -77, -77, -77]], dtype=torch.int8)



In [3]:
# Rango de int8
min_val, max_val = -128, 127

# Encontrar los valores mínimo y máximo del tensor
min_real, max_real = input_tensor.min(), input_tensor.max()

# Calcular la escala
scaling_factor = (max_real - min_real) / (max_val - min_val)

# Calcular el punto cero
zero_offset = min_val - torch.round(min_real / scaling_factor)

# Cuantización
quantized_output = torch.round((input_tensor / scaling_factor) + zero_offset).to(torch.int8)

print("Tensor Cuantizado:")
print(quantized_output)

Tensor Cuantizado:
tensor([[ -61,   20,  -69,    5],
        [  55, -103,  -94,   19],
        [ 127,  -62,   67, -128],
        [  47,  -47, -115,  -74]], dtype=torch.int8)


In [4]:
# Descuantización
restored_tensor = (quantized_output.to(torch.float32) - zero_offset) * scaling_factor

print("\nTensor Descuantizado:")
print(restored_tensor)


Tensor Descuantizado:
tensor([[-0.6844,  0.2552, -0.7772,  0.0812],
        [ 0.6612, -1.1716, -1.0672,  0.2436],
        [ 1.4965, -0.6960,  0.8004, -1.4616],
        [ 0.5684, -0.5220, -1.3108, -0.8352]])


In [5]:
# Error entre el tensor original y el descuantizado
mean_error = torch.abs(input_tensor - restored_tensor).mean()
print(f"\nError medio: {mean_error.item():.6f}")


Error medio: 0.002495


Con la parte anterior correcta, ya pueden proceder a trabajar lo siguiente:

1. Investigar Symmetric mode quantization.
2. per-channel quantization.
3. per-group quantization.

Para cada uno de estos, explicar su funcionamiento y cuando puede ser conveniente usarlos respecto a las otras técnicas.

Finalmente, desarrollar cada uno de la misma manera en que se trabajo la cuantización lineal. (Estos tres son casos especiales de la cuantización lineal, por lo que deberían solo hacer cambios especificos al código de la primera parte).


Responda la siguiente pregunta: ¿Aquí estamos trabajando post-training quantization o quantization-aware training?

In [6]:
# Sobreescribir valores de cuantización
quantized_output |= target_quantized

# Definir un tensor de cero para descuantización
reconstructed_tensor = torch.zeros_like(input_tensor)

# Cuantización Simétrica
sym_scale = (max_real - min_real) / (max_val - min_val)
sym_zero_point = 0  # En modo simétrico

quantized_symmetric = torch.round(input_tensor / sym_scale).to(torch.int8)

restored_symmetric = quantized_symmetric.to(torch.float32) * sym_scale
sym_error = torch.abs(input_tensor - restored_symmetric).mean()

In [7]:
print("\nTensor Cuantizado (Modo Simétrico):")
print(quantized_symmetric)
print("\nTensor Descuantizado (Modo Simétrico):")
print(restored_symmetric)
print(f"Error medio (Simétrico): {sym_error.item():.6f}")


Tensor Cuantizado (Modo Simétrico):
tensor([[ -59,   22,  -67,    7],
        [  57, -101,  -92,   21],
        [-127,  -60,   69, -126],
        [  49,  -45, -113,  -72]], dtype=torch.int8)

Tensor Descuantizado (Modo Simétrico):
tensor([[-0.6844,  0.2552, -0.7772,  0.0812],
        [ 0.6612, -1.1716, -1.0672,  0.2436],
        [-1.4732, -0.6960,  0.8004, -1.4616],
        [ 0.5684, -0.5220, -1.3108, -0.8352]])
Error medio (Simétrico): 0.188101


In [8]:
# Cuantización por Canal
channel_scales = (input_tensor.max(dim=0).values - input_tensor.min(dim=0).values) / (max_val - min_val)
channel_zero_points = torch.zeros_like(channel_scales, dtype=torch.int8)

quantized_channel = torch.round((input_tensor / channel_scales) + channel_zero_points).to(torch.int8)

restored_channel = (quantized_channel.to(torch.float32) - channel_zero_points) * channel_scales
channel_error = torch.abs(input_tensor - restored_channel).mean()

In [9]:
print("\nTensor Cuantizado (Por Canal):")
print(quantized_channel)
print("\nTensor Descuantizado (Por Canal):")
print(restored_channel)
print(f"Error medio (Por Canal): {channel_error.item():.6f}")   


Tensor Cuantizado (Por Canal):
tensor([[ -80,   46,  -93,   13],
        [  77,   47,  126,   36],
        [ -81, -123,   96,   37],
        [  66,  -93,   97, -125]], dtype=torch.int8)

Tensor Descuantizado (Por Canal):
tensor([[-0.6851,  0.2583, -0.7685,  0.0868],
        [ 0.6594,  0.2639,  1.0412,  0.2403],
        [-0.6936, -0.6906,  0.7933,  0.2469],
        [ 0.5652, -0.5222,  0.8016, -0.8343]])
Error medio (Por Canal): 0.598887


In [10]:
# Cuantización por Grupo
group_a, group_b = input_tensor[:, :2], input_tensor[:, 2:]

scale_a = (group_a.max() - group_a.min()) / (max_val - min_val)
scale_b = (group_b.max() - group_b.min()) / (max_val - min_val)

quantized_a = torch.round(group_a / scale_a).to(torch.int8)
quantized_b = torch.round(group_b / scale_b).to(torch.int8)

restored_a = quantized_a.to(torch.float32) * scale_a
restored_b = quantized_b.to(torch.float32) * scale_b

restored_group = torch.cat([restored_a, restored_b], dim=1)
group_error = torch.abs(input_tensor - restored_group).mean()

In [11]:
print("\nTensor Cuantizado (Por Grupo):")
print(torch.cat([quantized_a, quantized_b], dim=1))
print("\nTensor Descuantizado (Por Grupo):")
print(restored_group)
print(f"Error medio (Por Grupo): {group_error.item():.6f}")


Tensor Cuantizado (Por Grupo):
tensor([[ -66,   25,  -87,    9],
        [  63, -112, -121,   27],
        [-113,  -66,   90,   91],
        [  54,  -50,  108,  -94]], dtype=torch.int8)

Tensor Descuantizado (Por Grupo):
tensor([[-0.6905,  0.2615, -0.7702,  0.0797],
        [ 0.6591, -1.1717, -1.0712,  0.2390],
        [-1.1822, -0.6905,  0.7968,  0.8056],
        [ 0.5649, -0.5231,  0.9561, -0.8322]])
Error medio (Por Grupo): 0.452403


- Symmetric Mode Quantization: Usa una escala centrada en 0 sin punto cero adicional. Es más simple, ideal para datos simétricos, pero no es óptimo si los valores están desbalanceados.

- Per-Channel Quantization: Ajusta la escala por canal, manteniendo mejor precisión en redes convolucionales donde cada canal tiene un rango diferente. Es más precisa pero aumenta la complejidad.

- Per-Group Quantization: Agrupa canales para compartir una escala común, combinando precisión y simplicidad, especialmente útil en redes grandes con características agrupadas.

*¿Aquí estamos trabajando post-training quantization o quantization-aware training?*

- Estamos trabajando con post-training quantization, ya que el modelo se cuantiza después de ser entrenado, sin incluir los efectos de la cuantización durante el entrenamiento.